In [1]:
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
import time
####################you will need to change some paths here!#####################
#list of input files
dir_in='f:/data/tc_wakes/database/info/'
dir_out='f:/data/tc_wakes/database/sst/'
dir_mur = 'F:/data/sst/jpl_mur/v4.1/'
#output files
#filename_out_nc='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/data_so_far.nc'
#filename_out_kml='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/data_so_far.kml'
#################################################################################
import datetime as dt
import xarray as xr
from datetime import datetime
import pandas
import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import math
#from math import cos, radians

In [ ]:
dir_ccmp='F:/data/sat_data/ccmp/v02.0/Y'
date_1858 = dt.datetime(1858,11,17,0,0,0) # start date is 11/17/1958
dx=0.25
dy=0.25
dx_offset = -179.875
dy_offset = -78.3750


In [ ]:
###### dir_mur = 'F:/data/sst/jpl_mur/v4.1/'
for root, dirs, files in os.walk(dir_in, topdown=False):
#    for ii in range(12,13): 
    for name in files:
    #    name = files[ii]
#    for name in files:
        fname_in=os.path.join(root, name)
        fname_out=dir_out + fname_in[31:39] + '_all_25km.nc'
        inum_storm=int(fname_in[36:39])
        iyr_storm=int(fname_in[31:35])
        if iyr_storm>2003 or iyr_storm<2003:
            continue
#        if iyr_storm==2011 and inum_storm<15:
#            continue
        print(name,fname_in)
        dsx = xr.open_dataset(fname_in)
        lats = dsx.lat[0,:]
        lons = dsx.lon[0,:]  #lons goes from 0 to 360
        dysince = dsx.time
        #minlon=min(lons[0,:].values)-10
        #maxlon=max(lons[0,:].values)+10
        #minlat=min(lats[0,:].values)-10
        #maxlat=max(lats[0,:].values)+10
        
        iwrap=0
        minlon=min(lons.values)-10
        maxlon=max(lons.values)+10
        minlat=min(lats.values)-10
        maxlat=max(lats.values)+10
        if minlon<10 and maxlon>350:  #wrapping around meridion need to cal new min/max lon
            minlon=max(lons[lons<180].values)+10
            maxlon=min(lons[lons>180].values)-10
            iwrap=1 #set flag for wraparound
        
        #here is a fix for when a storm goes from 350 across 360 to 1 2 longitude
#        iwrap=0
#        print('first and last!',lons[0,1].values,lons[0,-1].values)
#        if abs(min(lons[0,:].values)-max(lons[0,:].values))>180:
#            lons1=lons[0,:].values-10>180
#            lons2=lons[0,:].values+10<180
#            maxlon=min(lons[0,lons1].values-10)
#            minlon=max(lons[0,lons2].values+10)
#            print('wrapped',minlon,maxlon)
#            iwrap=1
            #wrap_lons = ((lons+180) % 360) - 180        
            #maxlon=max(wrap_lons[0,:].values)+10 #this will find the positive maximum
            #minlon=min(wrap_lons[0,:].values)-10
            #if minlon<0:
            #    maxlon=min(wrap_lons[0,:].values)-10+360
            #    minlon=max(wrap_lons[0,:].values)+10

        print('min/max lon lat',minlon,maxlon,minlat,maxlat)

        ix1=int(round((minlon-dx_offset)/dx))
        ix2=int(round((maxlon-dx_offset)/dx))
        iy1=int(round((minlat-dy_offset)/dy))
        iy2=int(round((maxlat-dy_offset)/dy))
        if iy2 > 628:
            iy2=628
        if iy1 < 1:
            iy1=1    
        if ix1 < 0:
            ix1 = ix1 + 1440
        if ix2 < 0:
            ix2 = ix2 + 1440
        print(minlon,maxlon,minlat,maxlat)
        xdim=ix2-ix1
        if iwrap==1:  #wraps around so make sure xdim reflects that
            xdim=ix1-ix2+1440
        ydim=iy2-iy1
        
        dims=lats.shape
        tdim=dims[0]
        tem_date=[0]*tdim #print(dysince.values)
        for i in range(0,tdim):
            tem_date[i]=date_1858+dt.timedelta(days=float(dysince[0,i].values))  #create new time array that can be queried for year etc
        minjdy = min(tem_date).timetuple().tm_yday  #create new time array that can be queried for year etc
        minyear =min(tem_date).year #create new time array that can be queried for year etc
        maxjdy = max(tem_date).timetuple().tm_yday  #create new time array that can be queried for year etc
        maxyear =max(tem_date).year  #create new time array that can be queried for year etc
        print(minyear,minjdy,maxyear,maxjdy)
        
        dif = max(tem_date)-min(tem_date)
        tdim=int(dif.days)+30


        print(tdim,ix1,ix2,iy1,iy2,xdim,ydim)      
               
        sst_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        sst_clim_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        wndu_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        wndv_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        wndu_clim_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        wndv_clim_out_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        
        print('sst_out_sv',sst_out_sv.shape)
        for i in range(0,tdim):
            storm_date = tem_date[0]+dt.timedelta(days=i)
            #print(storm_date)
            
            syr=str(storm_date.year)
            smon=str(storm_date.month)
            sdym=str(storm_date.day)
            sjdy=str(storm_date.timetuple().tm_yday)
            fname_tem=syr + smon.zfill(2) + sdym.zfill(2) + '090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
            mur_filename = dir_mur + syr + '/' + sjdy.zfill(3) + '/' + fname_tem
            fname_tem='/CCMP_Wind_Analysis_' + syr + smon.zfill(2) + sdym.zfill(2) + '_V02.0_L3.0_RSS.nc'
            ccmp_filename = dir_ccmp + syr + '/M' + smon.zfill(2) + fname_tem      

            if storm_date.timetuple().tm_yday==366:
                sjdy = '365'
            clim_filename='F:/data/sst/jpl_mur/v4.1/clim/clim2_' + sjdy.zfill(3) +'_2003_2013_MUR-GLOB-v02.0-fv04.1.nc'
            ccmp_clim_filename='F:/data/sat_data/ccmp/v02.0/clim/ccmp_daily_clim_' + sjdy.zfill(3) +'.nc'
#            print(ccmp_filename)
#            print(ccmp_clim_filename)

#ccmp wind            
            nc_fid = Dataset(ccmp_filename, 'r')
            nc_fid2 = Dataset(ccmp_clim_filename, 'r')  
            tem = nc_fid.variables['uwnd'][:,iy1:iy2,:]
            tem = np.mean(tem,axis=0)
            wndu = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            tem = nc_fid.variables['vwnd'][:,iy1:iy2,:]
            tem = np.mean(tem,axis=0)
            wndv = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            mlat_ccmp = nc_fid.variables['latitude'][iy1:iy2]
            tem = nc_fid.variables['longitude'][:]
            mlon_ccmp = np.append(tem[ydim:],tem[:ydim], axis=0)               
            mlon_ccmp = ((mlon_ccmp - 180) % 360) - 180  #make -180 to 180 rather than 0 360
            tem = nc_fid2.variables['av_u'][iy1:iy2,:]
            wndu_clim = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            tem = nc_fid2.variables['av_v'][iy1:iy2,:]
            wndv_clim = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            nc_fid.close()
            nc_fid2.close()           
            
#            if i==0:
#                print('i=0',iy1,iy2,ix1,ix2,iy2-iy1,ix2-ix1)
            if ix1<=1440 and ix2<=1440 and iwrap==0:
#                if i==0:
#                    print('inside1',iy1,iy2,ix1,ix2)
                wndu_out = wndu[:,ix1:ix2]
                wndu_clim_out = wndu_clim[:,ix1:ix2]
                wndv_out = wndv[:,ix1:ix2]
                wndv_clim_out = wndv_clim[:,ix1:ix2]
            if ix1>1440 and ix2>1440 and iwrap==0:
 #               if i==0:
#                    print('inside2',iy1,iy2,ix1,ix2)
                wndu_out = wndu[:,ix1-1440:ix2-1440]
                wndu_clim_out = wndu_clim[:,ix1-1440:ix2-1440]
                wndv_out = wndv[:,ix1-1440:ix2-1440]
                wndv_clim_out = wndv_clim[:,ix1-1440:ix2-1440]
            if ix1<=1440 and ix2>1440 and iwrap==0:
#                if i==0:
#                    print('inside3',iy1,iy2,ix1,ix2)

                tem1 = wndu[:,ix1:]
                tem2 = wndu[:,:ix2-1440]
                wndu_out = np.append(tem1,tem2, axis=1)
                tem1 = wndv[:,ix1:]
                tem2 = wndv[:,:ix2-1440]
                wndv_out = np.append(tem1,tem2, axis=1)
                tem1 = wndu_clim[:,ix1:]
                tem2 = wndu_clim[:,:ix2-1440]
                wndu_clim_out = np.append(tem1,tem2, axis=1)
                tem1 = wndv_clim[:,ix1:]
                tem2 = wndv_clim[:,:ix2-1440]
                wndv_clim_out = np.append(tem1,tem2, axis=1)

            if ix1<=1440 and ix2>1440 and iwrap==1:
#                if i==0:
#                    print('inside1',iy1,iy2,ix1,ix2)
                wndu_out = wndu[:,ix2-1440:ix1]
                wndu_clim_out = wndu_clim[:,ix2-1440:ix1]
                wndv_out = wndv[:,ix2-1440:ix1]
                wndv_clim_out = wndv_clim[:,ix2-1440:ix1]

            wndu_out_sv[i,:,:]=wndu_out
            wndv_out_sv[i,:,:]=wndv_out
            wndu_clim_out_sv[i,:,:]=wndu_clim_out
            wndv_clim_out_sv[i,:,:]=wndv_clim_out
            
#sst data   

            nc_fid = Dataset(mur_filename, 'r')
            mlat = nc_fid.variables['lat'][1149:16849]
            ilat_mur1 = np.argmin(abs(mlat-mlat_ccmp.min()))-12
            #print('mlat first point:', mlat[ilat_mur1_tem])
            ilat_mur2 = np.argmin(abs(mlat-mlat_ccmp.max()))+13
            mlat = mlat[ilat_mur1:ilat_mur2]
            sst = nc_fid.variables['analysed_sst'][0,ilat_mur1:ilat_mur2,:]
            mlon = nc_fid.variables['lon'][:]
            nc_fid.close()

            nc_fid2 = Dataset(clim_filename, 'r')                      
            sst_clim = nc_fid2.variables['sst'][ilat_mur1:ilat_mur2,:]
            nc_fid2.close()
                       
            coarseness = 25
            temp = mlon.reshape((mlon.shape[0] // coarseness, coarseness))
            coarse_mlon = np.mean(temp, axis=(1), dtype=np.float64)
            temp = mlat.reshape((mlat.shape[0] // coarseness, coarseness))
            coarse_mlat = np.mean(temp, axis=(1), dtype=np.float64)
            temp = sst.reshape((sst.shape[0] // coarseness, coarseness, sst.shape[1] // coarseness, coarseness))
            coarse_sst = np.mean(temp, axis=(1,3), dtype=np.float64)
            temp = sst_clim.reshape((sst_clim.shape[0] // coarseness, coarseness, sst_clim.shape[1] // coarseness, coarseness))
            coarse_sst_clim = np.mean(temp, axis=(1,3), dtype=np.float64)

            #need to recalculate iy1 and iy2 because of offset made earlier to read less of file
#            iy1=np.argmin(abs(coarse_mlat-minlat))
#            iy2=np.argmin(abs(coarse_mlat-maxlat))
#            ydim=iy2-iy1   
#            print(coarse_mlat[0],coarse_mlat[-1])
#            print(iy1,iy2,ydim,minlat,maxlat)

#            if i==0:
#                print('i=0',ix1,ix2,iy2-iy1,ix2-ix1)
            if ix1<=1440 and ix2<=1440 and iwrap==0:
#                if i==0:
#                    print('inside1',ix1,ix2)
                sst_out = coarse_sst[:,ix1:ix2]
                mlat_out = coarse_mlat[:]
                mlon_out = coarse_mlon[ix1:ix2]
                sst_clim_out = coarse_sst_clim[:,ix1:ix2]
            if ix1>1440 and ix2>1440 and iwrap==0:
#                if i==0:
#                    print('inside2',ix1,ix2)
                sst_out = coarse_sst[:,ix1-1440:ix2-1440]
                mlat_out = coarse_mlat[:]
                mlon_out = coarse_mlon[ix1-1440:ix2-1440]
                sst_clim_out = coarse_sst_clim[:,ix1-1440:ix2-1440]
            if ix1<=1440 and ix2>1440 and iwrap==0:
#                if i==0:
#                    print('inside3',ix1,ix2)
                tem1 = coarse_sst[:,ix1:]
                tem2 = coarse_sst[:,:ix2-1440]
                sst_out = np.append(tem1,tem2, axis=1)
                mlat_out = coarse_mlat[:]
                mlon1 = coarse_mlon[ix1:]
                mlon2 = coarse_mlon[:ix2-1440]
                print(mlon1.shape,mlon2.shape)
                mlon_out = np.append(mlon1,mlon2, axis=0)               
                tem1 = coarse_sst_clim[:,ix1:]
                tem2 = coarse_sst_clim[:,:ix2-1440]
                sst_clim_out = np.append(tem1,tem2, axis=1)

            if ix1<=1440 and ix2>1440 and iwrap==1:
 #               if i==0:
 #                   print('inside1',ix1,ix2)
                sst_out = coarse_sst[:,ix2-1440:ix1]
                mlat_out = coarse_mlat[:]
                mlon_out = coarse_mlon[ix2-1440:ix1]
                sst_clim_out = coarse_sst_clim[:,ix2-1440:ix1]

            if i==0:
                print('sst',sst_out.shape,'coarse',coarse_sst.shape,wndu_out.shape)
                print('sst',sst_out.shape,'sst_sv',sst_out_sv.shape)
                print('mlon',mlon_out.shape,'mlat',mlat_out.shape)
            #sst_sv[i,:,:]=sst-sst_clim
            sst_out_sv[i,:,:]=sst_out
            sst_clim_out_sv[i,:,:]=sst_clim_out
            
        ilen=len(fname_in)
        
        dif_dys=[0]*tdim
        for i in range(0,tdim):
            dif_dys[i] = i

        print('file out:',fname_out)
        #f.close()
        f = Dataset(fname_out,'w', format='NETCDF4') 
        tempgrp = f.createGroup('data')
        tempgrp.setncattr_string('start time',str(tem_date[0]))
        tempgrp.createDimension('t', tdim)
        tempgrp.createDimension('y', ydim)
        tempgrp.createDimension('x', xdim)

    #tem_date[i]
        sst_netcdf = tempgrp.createVariable('sst', 'f4', ('t', 'y', 'x'))
        sst_clim_netcdf = tempgrp.createVariable('sst_clim', 'f4', ('t', 'y', 'x'))
        wndu_netcdf = tempgrp.createVariable('wndu', 'f4', ('t', 'y', 'x'))
        wndv_netcdf = tempgrp.createVariable('wndv', 'f4', ('t', 'y', 'x'))
        wndu_clim_netcdf = tempgrp.createVariable('wndu_clim', 'f4', ('t', 'y', 'x'))
        wndv_clim_netcdf = tempgrp.createVariable('wndv_clim', 'f4', ('t', 'y', 'x'))
        longitude = tempgrp.createVariable('lon', 'f4', 'x')
        latitude = tempgrp.createVariable('lat', 'f4', 'y')  
        time = tempgrp.createVariable('time', 'i4', 't')
        
        sst_netcdf[:] = sst_out_sv
        sst_clim_netcdf[:] = sst_clim_out_sv
        wndu_netcdf[:] = wndu_out_sv
        wndv_netcdf[:] = wndv_out_sv
        wndu_clim_netcdf[:] = wndu_clim_out_sv
        wndv_clim_netcdf[:] = wndv_clim_out_sv
        latitude[:] = mlat_out
        longitude[:] = mlon_out
        time[:]=dif_dys
        f.close()
    


002annual_storm_info.nc f:/data/tc_wakes/database/info/2003\002annual_storm_info.nc
min/max lon lat 38.49999891594052 88.29999824985862 -25.499999653548002 3.800000138580799
38.49999891594052 88.29999824985862 -25.499999653548002 3.800000138580799
2002 318 2002 331
42 873 1073 212 329 200 117
sst_out_sv (42, 117, 200)
sst (117, 200) coarse (117, 1440) (117, 200)
sst (117, 200) sst_sv (42, 117, 200)
mlon (200,) mlat (117,)
file out: f:/data/tc_wakes/database/sst/2003\002_all_25km.nc
003annual_storm_info.nc f:/data/tc_wakes/database/info/2003\003annual_storm_info.nc
min/max lon lat 171.60000398755074 236.0000029951334 -47.49999916180968 -6.299999635666609
171.60000398755074 236.0000029951334 -47.49999916180968 -6.299999635666609
2002 337 2002 345
37 1406 1664 124 288 258 164
sst_out_sv (37, 164, 258)
(34,) (224,)
sst (164, 258) coarse (164, 1440) (164, 258)
sst (164, 258) sst_sv (37, 164, 258)
mlon (258,) mlat (164,)
(34,) (224,)
(34,) (224,)
(34,) (224,)
(34,) (224,)
(34,) (224,)
(34,) 

(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
(0,) (109,)
file out: f:/data/tc_wakes/database/sst/2003\017_all_25km.nc
018annual_storm_info.nc f:/data/tc_wakes/database/info/2003\018annual_storm_info.nc
min/max lon lat 43.499998804181814 86.09999829903245 -37.99999937415123 -1.2999997474253178
43.499998804181814 86.09999829903245 -37.99999937415123 -1.2999997474253178
2003 36 2003 49
43 893 1064 162 308 171 146
sst_out_sv (43, 146, 171)
sst (146, 171) coarse (146, 1440) (146, 171)
sst (146, 171) sst_sv (43, 146, 171)
mlon (171,) mlat (146,)
file out: f:/data/tc_wakes/database/sst/200

In [36]:
            nc_fid = Dataset(ccmp_filename, 'r')
            nc_fid2 = Dataset(ccmp_clim_filename, 'r')  
            tem = nc_fid.variables['uwnd'][:,iy1:iy2,:]
            tem = np.mean(tem,axis=0)
            wndu = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            tem = nc_fid.variables['vwnd'][:,iy1:iy2,:]
            tem = np.mean(tem,axis=0)
            wndv = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            mlat_ccmp = nc_fid.variables['latitude'][iy1:iy2]
            tem = nc_fid.variables['longitude'][:]
            mlon_ccmp = np.append(tem[ydim:],tem[:ydim], axis=0)               
            mlon_ccmp = ((mlon_ccmp - 180) % 360) - 180  #make -180 to 180 rather than 0 360
            tem = nc_fid2.variables['av_u'][iy1:iy2,:]
            wndu_clim = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            tem = nc_fid2.variables['av_v'][iy1:iy2,:]
            wndv_clim = np.append(tem[:,ydim:],tem[:,:ydim], axis=1)               
            nc_fid.close()
            nc_fid2.close()           

print(tem.shape)
print(wndu_clim.shape)
mlat_ccmp[0:5]

(165, 1440)
(165, 1440)


masked_array(data=[-77.625, -77.375, -77.125, -76.875, -76.625],
             mask=False,
       fill_value=1e+20,
            dtype=float32)

In [52]:
            nc_fid = Dataset(mur_filename, 'r')
            mlat = nc_fid.variables['lat'][1149:16849]
            ilat_mur1 = np.argmin(abs(mlat-mlat_ccmp.min()))-12
            print('mlat first point:', mlat[ilat_mur1_tem])
            ilat_mur2 = np.argmin(abs(mlat-mlat_ccmp.max()))+13
            mlat = mlat[ilat_mur1:ilat_mur2]
            nc_fid.close()
            coarseness = 25
            temp = mlat.reshape((mlat.shape[0] // coarseness, coarseness))
            coarse_mlat = np.mean(temp, axis=(1), dtype=np.float64)

print('averaged mlat:', coarse_mlat[0:5])
print('average ccmp:', mlat_ccmp[0:5])
print('averaged mlat:', coarse_mlat[-2:-1])
print('average ccmp:', mlat_ccmp[-2:-1])

mlat first point: -77.75
averaged mlat: [-77.63 -77.38 -77.13 -76.88 -76.63]
average ccmp: [-77.625 -77.375 -77.125 -76.875 -76.625]
averaged mlat: [-36.88]
average ccmp: [-36.875]


In [26]:
print(wndv_out.shape,sst_out.shape)
coarse_mlat

(164, 258) (165, 258)


masked_array(data=[-48.26      , -48.01      , -47.76      , -47.51      ,
                   -47.26      , -47.01      , -46.76      , -46.51      ,
                   -46.26      , -46.01      , -45.76      , -45.51      ,
                   -45.26      , -45.01      , -44.76      , -44.51      ,
                   -44.26      , -44.01      , -43.76      , -43.51      ,
                   -43.26      , -43.01      , -42.76      , -42.51      ,
                   -42.26      , -42.01      , -41.76      , -41.51      ,
                   -41.26      , -41.01      , -40.76      , -40.51      ,
                   -40.26      , -40.01      , -39.76      , -39.51      ,
                   -39.26      , -39.01      , -38.76      , -38.51      ,
                   -38.26      , -38.01      , -37.76      , -37.51      ,
                   -37.26      , -37.01      , -36.76      , -36.51      ,
                   -36.26      , -36.01      , -35.76      , -35.51      ,
                   -35.26

In [22]:
iy1=int(round((minlat-dy_offset)/dy))
iy2=int(round((maxlat-dy_offset)/dy))
if iy2 > 628:
    iy2=628
if iy1 < 1:
    iy1=1    
ydim=iy2-iy1
print(mlat_ccmp[iy1],mlat_ccmp[iy2])
iy1=np.argmin(abs(coarse_mlat-minlat))
iy2=np.argmin(abs(coarse_mlat-maxlat))
ydim=iy2-iy1   
#print(coarse_mlat[0],coarse_mlat[-1])
#print(iy1,iy2,ydim,minlat,maxlat)

print(mlat_out[1],mlat_out[-1])

-36.875 2.125
-36.53 1.72


In [23]:
            nc_fid = Dataset(mur_filename, 'r')
            mlat = nc_fid.variables['lat'][1149:16849]
            ilat_mur1_tem = np.argmin(abs(mlat-minlat))-100
            ilat_mur1 = np.argmin(abs(mlat-(round((mlat[ilat_mur1_tem]+89.875)/.25)*.25-89.875)))
            ilat_mur2 = np.argmin(abs(mlat-maxlat))+100
            int_25=math.ceil((ilat_mur2-ilat_mur1)/25)
            ilat_mur2=ilat_mur1+25*int_25
            mlat = mlat[ilat_mur1:ilat_mur2]
            sst = nc_fid.variables['analysed_sst'][0,ilat_mur1:ilat_mur2,:]
            mlon = nc_fid.variables['lon'][:]
            nc_fid.close()

            nc_fid2 = Dataset(clim_filename, 'r')                      
            sst_clim = nc_fid2.variables['sst'][ilat_mur1:ilat_mur2,:]
            nc_fid2.close()
                       
            coarseness = 25
            temp = mlon.reshape((mlon.shape[0] // coarseness, coarseness))
            coarse_mlon = np.mean(temp, axis=(1), dtype=np.float64)
            temp = mlat.reshape((mlat.shape[0] // coarseness, coarseness))
            coarse_mlat = np.mean(temp, axis=(1), dtype=np.float64)
            temp = sst.reshape((sst.shape[0] // coarseness, coarseness, sst.shape[1] // coarseness, coarseness))
            coarse_sst = np.mean(temp, axis=(1,3), dtype=np.float64)
            temp = sst_clim.reshape((sst_clim.shape[0] // coarseness, coarseness, sst_clim.shape[1] // coarseness, coarseness))
            coarse_sst_clim = np.mean(temp, axis=(1,3), dtype=np.float64)

            #need to recalculate iy1 and iy2 because of offset made earlier to read less of file
            iy1=np.argmin(abs(coarse_mlat-minlat))
            iy2=np.argmin(abs(coarse_mlat-maxlat))
            ydim=iy2-iy1   
            print(coarse_mlat[0],coarse_mlat[-1])
            print(iy1,iy2,ydim,minlat,maxlat)


-37.76 2.99
3 159 156 -36.899999398738146 2.0000001788139343
